<a href="https://colab.research.google.com/github/RafsanJany-44/Rhythms_off_Algorithms/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

#Data Set

In [2]:
!pip install tensorflow_datasets

#GPU setup

In [3]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [4]:

tf.config.list_physical_devices("GPU")
tf.test.gpu_device_name()
# Standard output is '/device:GPU:0'
gpus = tf.config.list_physical_devices("GPU")

#tf.config.experimental.set_memory_growth(gpus[0],True)

#tf.config.experimental.set_visible_devices(gpus[0],'GPU')
#logical_gpus= tf.config.experimental.list_logical_devices('GPU')
#print(len(gpus),"Physical GPUs", len(logical_gpus),"Logical GPU")

#Available Data set

In [5]:
tfds.list_builders()

['abstract_reasoning',
 'accentdb',
 'aeslc',
 'aflw2k3d',
 'ag_news_subset',
 'ai2_arc',
 'ai2_arc_with_ir',
 'amazon_us_reviews',
 'anli',
 'arc',
 'bair_robot_pushing_small',
 'bccd',
 'beans',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'blimp',
 'bool_q',
 'c4',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_1',
 'cifar10_corrupted',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'clic',
 'clinc_oos',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coco_captions',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'common_voice',
 'coqa',
 'cos_e',
 'cosmos_qa',
 'covid19sum',
 'crema_d',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'deep_weeds',
 'definite_pronoun_resolution',
 'dementiabank',
 'diabetic_retinopathy_detection',
 'div2k',
 'dmlab',
 'downsampled_imagenet',
 

In [6]:
tfds.builder("imdb_reviews").info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [7]:
imdb_train,ds_info=tfds.load(name="imdb_reviews",split="train",with_info=True,as_supervised=True)
imdb_test = tfds.load(name="imdb_reviews",split="test",as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete7D54MK/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete7D54MK/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [8]:
type(imdb_train)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [9]:
for example,label in imdb_train.take(3):
  print(example,"\n",label)

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string) 
 tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on

In [10]:
tokenizer = tfds.deprecated.text.Tokenizer()
vocabulary_set = set()

for example,label in imdb_train:
  some_token=tokenizer.tokenize(example.numpy())
  vocabulary_set.update(some_token)

In [11]:
some_token

['First',
 'of',
 'all',
 'Riget',
 'is',
 'wonderful',
 'Good',
 'comedy',
 'and',
 'mystery',
 'thriller',
 'at',
 'the',
 'same',
 'time',
 'Nice',
 'combination',
 'of',
 'strange',
 'dogma',
 'style',
 'of',
 'telling',
 'the',
 'story',
 'together',
 'with',
 'good',
 'music',
 'and',
 'great',
 'actors',
 'But',
 'unfortunately',
 'there',
 's',
 'no',
 'the',
 'end',
 'As',
 'for',
 'me',
 'it',
 's',
 'unacceptable',
 'I',
 'was',
 'thinking',
 'how',
 'it',
 'will',
 'be',
 'possible',
 'to',
 'continue',
 'the',
 'story',
 'without',
 'Helmer',
 'and',
 'Drusse',
 'and',
 'I',
 'have',
 'some',
 'idea',
 'I',
 'think',
 'Lars',
 'should',
 'make',
 'RIGET',
 'III',
 'a',
 'little',
 'bit',
 'different',
 'I',
 'm',
 'sure',
 'that',
 '3rd',
 'part',
 'without',
 'Helmer',
 'wouldn',
 't',
 'be',
 'the',
 'same',
 'So',
 'here',
 's',
 'my',
 'suggestion',
 'Mayble',
 'little',
 'bit',
 'stupid',
 'maybe',
 'not',
 'I',
 'know',
 'that',
 'Lars',
 'likes',
 'to',
 'experiment

In [12]:
tokenizer = tfds.deprecated.text.Tokenizer()
vocabulary_set = set()
MAX_TOKENS=0

for example,label in imdb_train:
  some_token=tokenizer.tokenize(example.numpy())
  if MAX_TOKENS < len(some_token):
    MAX_TOKENS=len(some_token)
  vocabulary_set.update(some_token)

In [13]:
vocabulary_set

{'MANY',
 'moolah',
 'ASAP',
 'Litel',
 'opus',
 'instinctual',
 'refried',
 'Montenegrin',
 'robed',
 'ingrained',
 'Wills',
 'contention',
 'leetle',
 'Halo',
 'loafers',
 'radical',
 'Beahan',
 'Princess',
 'castration',
 'hated',
 'couple',
 'Cube',
 'Calvinist',
 'thooughly',
 'collaborators',
 'Skullduggery',
 'youth',
 'Columbus',
 'Popoff',
 'KERR',
 'MBNA',
 'Chojnacki',
 'Samundar',
 'hypothetical',
 'Pain',
 'Linear',
 'lofts',
 'interpreters',
 'reily',
 'CAPE',
 'Obnoxious',
 'Rouge',
 'vocalize',
 'zigzaggy',
 'tinting',
 'rarity',
 'Nogerelli',
 'Kefauver',
 'tuck',
 'dust',
 'judgements',
 'unbelieveably',
 'Craziness',
 'expen',
 'MGR',
 'Sibrel',
 'pornoes',
 'Cheapness',
 'Unending',
 'intermixed',
 'Amusingly',
 'Imtiaz',
 'hyuk',
 'sneered',
 'trodden',
 'shootin',
 'Rogers',
 'random',
 'allow',
 'mysteriously',
 'picks',
 'recertified',
 'windshields',
 'smooth',
 'placements',
 'Forest',
 'downers',
 'Hardie',
 'move',
 'logging',
 'Borders',
 'EVEN',
 'overflow

In [14]:
type(vocabulary_set)

set

In [15]:
len(vocabulary_set)

93929

In [16]:
encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set, tokenizer=tokenizer)
sample_string = "Hello Dear How are you"
              
enocoded_string=encoder.encode(sample_string)
print('Encoded string is {}',format(enocoded_string))
                                                
orginal_string=encoder.decode(enocoded_string)
print('Orginal string is {}',format(orginal_string))


Encoded string is {} [17489, 81534, 88572, 63676, 88962]
Orginal string is {} Hello Dear How are you


In [17]:
imbd_encoder= encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set, tokenizer=tokenizer)

vocab_size=imbd_encoder.vocab_size

print(vocab_size,MAX_TOKENS)

93931 2525


In [19]:
for example, label in imdb_train.take(1):
  print(example)
  encoded=imbd_encoder.encode(example.numpy())

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)


In [20]:
type(encoded)

list

In [22]:
encoded

[36771,
 13653,
 15054,
 44075,
 74861,
 66828,
 13389,
 11536,
 29939,
 31704,
 72492,
 27818,
 35966,
 92696,
 16315,
 17223,
 5425,
 80945,
 63676,
 57527,
 81105,
 62723,
 57648,
 16082,
 56170,
 29939,
 13814,
 16625,
 16794,
 72492,
 33255,
 31190,
 13814,
 57527,
 86318,
 81485,
 8292,
 17210,
 57648,
 66828,
 26489,
 39593,
 71827,
 36771,
 66828,
 23077,
 15054,
 14202,
 47830,
 76022,
 32768,
 85211,
 34821,
 75269,
 73653,
 70361,
 10887,
 24796,
 85050,
 81119,
 47072,
 36392,
 10887,
 18464,
 13814,
 69515,
 32759,
 65580,
 3762,
 10750,
 91998,
 70535,
 22841,
 46301,
 68758,
 36801,
 25406,
 81221,
 37075,
 92696,
 13653,
 85549,
 62723,
 40730,
 73653,
 82075,
 62499,
 72492,
 40730,
 66828,
 79749,
 13653,
 11390,
 31851,
 51829,
 81528,
 36506,
 86564,
 90369,
 86097,
 79749,
 59857,
 63676,
 5455,
 10839,
 57648,
 67629,
 82253,
 26489,
 10839,
 35966,
 92696,
 26489,
 44250,
 60910,
 86564,
 81485,
 22861,
 68938,
 76454,
 78150]

In [21]:
print(imbd_encoder.decode(encoded))

This was an absolutely terrible movie Don t be lured in by Christopher Walken or Michael Ironside Both are great actors but this must simply be their worst role in history Even their great acting could not redeem this movie s ridiculous storyline This movie is an early nineties US propaganda piece The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions Maria Conchita Alonso appeared phony and her pseudo love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning I am disappointed that there are movies like this ruining actor s like Christopher Walken s good name I could barely sit through it


In [32]:
from tensorflow.keras.preprocessing import sequence

def encode_pad_transform(sample):
  encoded = imbd_encoder.encode(sample.numpy())
  pad = sequence.pad_sequences([encoded],padding='post',maxlen=150)
  return np.array(pad[0],dtype=np.int64)

def encode_tf_fn(sample,label):
  encoded = tf.py_function(encode_pad_transform,inp=[sample],Tout =(tf.int64))
  encoded.set_shape([None])
  label.set_shape([])
  return encoded, label


In [33]:
#test the transformation on a small subset
subset=imdb_train.take(10)
tst=subset.map(encode_tf_fn)

In [34]:
tst

<MapDataset shapes: ((None,), ()), types: (tf.int64, tf.int64)>

In [35]:
 for review,label in tst.take(1):
   print(review, "\n",label)
   print(imbd_encoder.decode(review))

tf.Tensor(
[36771 13653 15054 44075 74861 66828 13389 11536 29939 31704 72492 27818
 35966 92696 16315 17223  5425 80945 63676 57527 81105 62723 57648 16082
 56170 29939 13814 16625 16794 72492 33255 31190 13814 57527 86318 81485
  8292 17210 57648 66828 26489 39593 71827 36771 66828 23077 15054 14202
 47830 76022 32768 85211 34821 75269 73653 70361 10887 24796 85050 81119
 47072 36392 10887 18464 13814 69515 32759 65580  3762 10750 91998 70535
 22841 46301 68758 36801 25406 81221 37075 92696 13653 85549 62723 40730
 73653 82075 62499 72492 40730 66828 79749 13653 11390 31851 51829 81528
 36506 86564 90369 86097 79749 59857 63676  5455 10839 57648 67629 82253
 26489 10839 35966 92696 26489 44250 60910 86564 81485 22861 68938 76454
 78150     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0], shape=(150,), dtype=int64) 
 tf.Tensor(0, shape=(), dtype=int64)
This was 

In [37]:
#tokenize.encode/pad all training
# and tetrsing data
encoded_train=imdb_train.map(encode_tf_fn,num_parallel_calls=tf.data.experimental.AUTOTUNE)
encoded_test=imdb_test.map(encode_tf_fn,num_parallel_calls=tf.data.experimental.AUTOTUNE)

#preparing the model

In [39]:
#length of the vocabulary in chars
vocab_size=imbd_encoder.vocab_size

# The embedding dimension
embedding_dim=64

#NUmber of RNN units, positive integer, dimensionality of the output space
rnn_units=64

#batch size
BATCH_SIZE=100

In [40]:
def build_model_lstm(vocab_size,embedding_dim,rnn_units,batch_size):
  model = tf.keras.Sequential([
                tf.keras.layers.Embedding(vocab_size,embedding_dim,mask_zero=True,batch_input_shape=[batch_size,None]),
                tf.keras.layers.LSTM(rnn_units),
                tf.keras.layers.Dense(1,activation="sigmoid")
  ])
  return model

In [43]:
model= build_model_lstm(vocab_size=vocab_size, embedding_dim=embedding_dim,
                       rnn_units=rnn_units,batch_size=BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (100, None, 64)           6011584   
_________________________________________________________________
lstm (LSTM)                  (100, 64)                 33024     
_________________________________________________________________
dense (Dense)                (100, 1)                  65        
Total params: 6,044,673
Trainable params: 6,044,673
Non-trainable params: 0
_________________________________________________________________


In [44]:
model.save('model.h5')

In [45]:
model.compile(loss='binary_crosentropy',optimizer='adam',metrics=['accuracy','Precision','Recall'])

#Model training and evalution

In [46]:
#Prefetch for performanence
encoded_train_batched=encoded_train.batch(BATCH_SIZE).prefetch(100)

In [47]:
model.fit(encoded_train.batch(BATCH_SIZE),epochs=10)

Epoch 1/10


ValueError: ignored